<table align="left">

  <h1> Options to run the notebook </h1>
 
  <td>
    <a href="https://colab.research.google.com/github/Dong-Xuyong/Aprendizagem/blob/main/Projeto/bimbo-feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/Dong-Xuyong/Aprendizagem/blob/main/Projeto/bimbo-feature-engineering.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Set Up

In [1]:
import urllib.request
import zipfile
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#Imortar os dataset no codigo
url = "https://raw.githubusercontent.com/Dong-Xuyong/Aprendizagem/main/Projeto/Dataset/iniTables.zip"

urllib.request.urlretrieve(url, "df.zip")

#Extrair os datasets
with zipfile.ZipFile("df.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [3]:
df_final = pd.read_csv("0.df_final.csv")
df_cliente = pd.read_csv("1.cliente_tabla.csv")
df_produto = pd.read_csv("2.producto_tabla.csv")
df_address = pd.read_csv("3.2.address_filtered.csv")

# Data Modeling
On this section we will extract and filtered the Cliente_tabla and producto_tabla

## Client

In [4]:
df_cliente.head()

,Cliente_ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT


In [5]:
#Meter todos nomes dos cliente em maiusculas, tornar a nomes menos sensiveis
df_cliente['NombreCliente'] = df_cliente['NombreCliente'].str.upper()

#Drop dos clientes duplicados a partir do Cliente_ID
df_cliente = df_cliente.drop_duplicates(subset = "Cliente_ID")

In [6]:
def filter_specific(df):
    
    # Filtrar os clientes para empresas conhecidas
    
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*REMISION.*','Consignment')
    df['NombreCliente'] = df['NombreCliente'].replace(['.*WAL MART.*','.*SAMS CLUB.*'],'Walmart', regex=True)
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*OXXO.*','Oxxo Store')
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*CONASUPO.*','Govt Store')
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*BIMBO.*','Bimbo Store')
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*WALMART.*','Walmart Store')
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*COSTCO.*','Costco Store')
    

    # General term search for a random assortment of words I picked from looking at
    # their frequency of appearance in the data and common spanish words for these categories
    df['NombreCliente'] = df['NombreCliente'].replace(['.*COLEG.*','.*UNIV.*','.*ESCU.*','.*INSTI.*',\
                                                        '.*PREPAR.*'],'School', regex=True)
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*PUESTO.*','Post')
    df['NombreCliente'] = df['NombreCliente'].replace(['.*FARMA.*','.*HOSPITAL.*','.*CLINI.*'],'Hospital/Pharmacy', regex=True)
    df['NombreCliente'] = df['NombreCliente'].replace(['.*CAFE.*','.*CREMERIA.*','.*DULCERIA.*',\
                                                        '.*REST.*','.*BURGER.*','.*TACO.*', '.*TORTA.*',\
                                                        '.*TAQUER.*','.*HOT DOG.*',\
                                                        '.*COMEDOR.*', '.*ERIA.*','.*BURGU.*'],'Eatery', regex=True)
    df['NombreCliente'] = df['NombreCliente'].str.replace('.*SUPER.*','Supermarket')
    df['NombreCliente'] = df['NombreCliente'].replace(['.*COMERCIAL.*','.*BODEGA.*','.*DEPOSITO.*',\
                                                            '.*ABARROTES.*','.*MERCADO.*','.*CAMBIO.*',\
                                                        '.*MARKET.*','.*MART .*','.*MINI .*',\
                                                        '.*PLAZA.*','.*MISC.*','.*ELEVEN.*','.*EXP.*',\
                                                         '.*SNACK.*', '.*PAPELERIA.*', '.*CARNICERIA.*',\
                                                         '.*LOCAL.*','.*COMODIN.*','.*PROVIDENCIA.*'
                                                        ],'General Market/Mart'\
                                                       , regex=True)

    df['NombreCliente'] = df['NombreCliente'].replace(['.*VERDU.*','.*FRUT.*'],'Fresh Market', regex=True)
    df['NombreCliente'] = df['NombreCliente'].replace(['.*HOTEL.*','.*MOTEL.*'],'Hotel', regex=True)

In [7]:
# --- Begin filtering for more general terms
# The idea here is to look for names with particles of speech that would
# not appear in a person's name.
# i.e. "Individuals" should not contain any participles or numbers in their names.
    #Nao sao nomes de pessoas
def filter_participle(df):
    
    df['NombreCliente'] = df['NombreCliente'].replace([
            '.*LA .*','.*EL .*','.*DE .*','.*LOS .*','.*DEL .*','.*Y .*', '.*SAN .*', '.*SANTA .*',\
            '.*AG .*','.*LAS .*','.*MI .*','.*MA .*', '.*II.*', '.*[0-9]+.*'\
    ],'Small Franchise', regex=True)

In [8]:
# Any remaining entries should be "Individual" Named Clients, there are some outliers.
# More specific filters could be used in order to reduce the percentage of outliers in this final set.
def filter_remaining(df):
    def function_word(data):
        # Avoid the single-words created so far by checking for upper-case
        if (data.isupper()) and (data != "NO IDENTIFICADO"): 
            return 'Individual'
        else:
            return data
        
    #filtro da funcao filter_remaining na coluna NombreCliente por map (linha por linha)
    df['NombreCliente'] = df['NombreCliente'].map(function_word)

In [9]:
filter_specific(df_cliente)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of reg

In [10]:
filter_participle(df_cliente)

In [11]:
filter_remaining(df_cliente)

In [12]:
df_cliente['NombreCliente'].value_counts()

Individual             351884
NO IDENTIFICADO        281670
Small Franchise        158349
General Market/Mart     65512
Eatery                  30277
Supermarket             15907
Oxxo Store               9276
Hospital/Pharmacy        5693
School                   5562
Post                     2657
Hotel                    1104
Fresh Market             1060
Govt Store                958
Bimbo Store               319
Walmart                   217
Costco Store               32
Consignment                14
Walmart Store               9
Name: NombreCliente, dtype: int64

In [13]:
df_produto.head()

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


## Product

In [14]:
#filtrar o nome do produto
df_produto['short_name'] = df_produto.NombreProducto.str.extract('^(\D*)', expand=False)

#filtar a marca do produto
df_produto['brand'] = df_produto.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)

#filtrar o peso do producto
w = df_produto.NombreProducto.str.extract('(\d+)([gG]|[kK][gG]|[mM][lL])', expand=True)

#transfomar o peso a partir de w[0] (o valor) * w[1] medida em gramas
df_produto['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'kg':1000, 'g':1, 'ml':1})

#filtar os pacotes dos produtos
df_produto['pieces'] =  df_produto.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

In [15]:
df_produto.head()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750.0,NaN
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170.0,NaN
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45.0,4.0


In [16]:

df_produto[df_produto['weight'].isnull()].iloc[:10]

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN
40,322,Tuinky Fresas con Crema 2p WON 322,Tuinky Fresas con Crema,WON,NaN,2.0
93,1166,Bollo Regular 2pq 30p McD BIM 1166,Bollo Regular,BIM,NaN,30.0
117,1289,Donas 6P Prom BIM 1289,Donas,BIM,NaN,NaN
127,1641,Doraditas 4p TR 1641,Doraditas,TR,NaN,4.0
190,3509,Paletina para Cafe NES 3509,Paletina para Cafe NES,NES,NaN,NaN
199,3609,Mantecadas Chocolate 3p TR 3609,Mantecadas Chocolate,TR,NaN,3.0
232,4240,Mantecadas 2p SP TR 4240,Mantecadas,TR,NaN,2.0
270,5345,Rollo Fresa 2p 75G MLA 5345,Rollo Fresa,MLA,NaN,2.0
296,5707,Tostada Ondulada Tubo 30p MR 5707,Tostada Ondulada Tubo,MR,NaN,30.0


In [17]:
df_produto.size

15552

In [18]:

df_produto.isnull().sum()

Producto_ID          0
NombreProducto       0
short_name           0
brand                1
weight              57
pieces            1475
dtype: int64

In [19]:
df_produto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Producto_ID     2592 non-null   int64  
 1   NombreProducto  2592 non-null   object 
 2   short_name      2592 non-null   object 
 3   brand           2591 non-null   object 
 4   weight          2535 non-null   float64
 5   pieces          1117 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 121.6+ KB


## Join tables
The coordinates of the address was done by fist joing the State and Town columns and then extract on Geolocation API by Google

In [20]:
# juncao da table town state com df_final
df_town = pd.read_csv("3.town_state.csv")
df_final = pd.merge(df_final, df_town, on="Agencia_ID", how="inner")

In [21]:
#concatenar as columas state e town.
df_final['Address'] = df_final[['State', 'Town']].agg(', '.join, axis=1)

In [22]:
#Juntar todas as tabelas ao df_final
df_main = pd.merge(df_final, df_cliente, on="Cliente_ID", how="inner")
df_main = pd.merge(df_main, df_address, on="Address", how="inner")
df_main = pd.merge(df_main, df_produto, on="Producto_ID", how="inner")

In [23]:
df_main.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State,Address,NombreCliente,Latitude,Longitude,NombreProducto,short_name,brand,weight,pieces
0,5,1936,1,1158,1575677,2425,60,270.0,0,0.0,...,GUANAJUATO,"GUANAJUATO, 2300 LEON MILENIUM",Small Franchise,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0
1,3,1936,1,1164,4511404,2425,60,270.0,0,0.0,...,GUANAJUATO,"GUANAJUATO, 2300 LEON MILENIUM",NO IDENTIFICADO,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0
2,4,1936,1,1161,66167,2425,24,108.0,0,0.0,...,GUANAJUATO,"GUANAJUATO, 2300 LEON MILENIUM",Individual,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0
3,4,1936,1,1164,948201,2425,1,4.5,2,9.0,...,GUANAJUATO,"GUANAJUATO, 2300 LEON MILENIUM",Individual,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0
4,3,1936,1,1156,877954,2425,55,247.5,0,0.0,...,GUANAJUATO,"GUANAJUATO, 2300 LEON MILENIUM",Individual,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0


In [24]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 964348 entries, 0 to 964347
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Semana             964348 non-null  int64  
 1   Agencia_ID         964348 non-null  int64  
 2   Canal_ID           964348 non-null  int64  
 3   Ruta_SAK           964348 non-null  int64  
 4   Cliente_ID         964348 non-null  int64  
 5   Producto_ID        964348 non-null  int64  
 6   Venta_uni_hoy      964348 non-null  int64  
 7   Venta_hoy          964348 non-null  float64
 8   Dev_uni_proxima    964348 non-null  int64  
 9   Dev_proxima        964348 non-null  float64
 10  Demanda_uni_equil  964348 non-null  int64  
 11  Town               964348 non-null  object 
 12  State              964348 non-null  object 
 13  Address            964348 non-null  object 
 14  NombreCliente      964348 non-null  object 
 15  Latitude           964348 non-null  float64
 16  Lo

In [25]:
#contar os valor da Ruta_SAK para obter as rotas mais frequentadas
df_ = df_main['Ruta_SAK'].value_counts().rename_axis('Ruta_SAK').reset_index(name='Ruta_SAK_counts')

In [26]:
df_

,Ruta_SAK,Ruta_SAK_counts
0,1201,6021
1,1203,5882
2,1204,5553
3,1202,5501
4,1205,5336
...,...,...
2259,7734,1
2260,8476,1
2261,7453,1
2262,7988,1


In [27]:

df_main = pd.merge(df_main, df_, on='Ruta_SAK', how='inner')

In [28]:
df_main.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Address,NombreCliente,Latitude,Longitude,NombreProducto,short_name,brand,weight,pieces,Ruta_SAK_counts
0,5,1936,1,1158,1575677,2425,60,270.0,0,0.0,...,"GUANAJUATO, 2300 LEON MILENIUM",Small Franchise,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0,2008
1,8,1936,1,1158,841050,2425,20,90.0,0,0.0,...,"GUANAJUATO, 2300 LEON MILENIUM",Individual,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0,2008
2,6,1936,1,1158,3422113,2425,10,45.0,0,0.0,...,"GUANAJUATO, 2300 LEON MILENIUM",NO IDENTIFICADO,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0,2008
3,7,1936,1,1158,1570783,2425,25,112.5,0,0.0,...,"GUANAJUATO, 2300 LEON MILENIUM",Small Franchise,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0,2008
4,9,1936,1,1158,2296923,2425,23,103.5,0,0.0,...,"GUANAJUATO, 2300 LEON MILENIUM",Small Franchise,21.122129,-101.60347,Nito 1p 62g Central BIM 2425,Nito,BIM,62.0,1.0,2008


In [29]:
df_main.columns

Index(['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID',
       'Producto_ID', 'Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima',
       'Dev_proxima', 'Demanda_uni_equil', 'Town', 'State', 'Address',
       'NombreCliente', 'Latitude', 'Longitude', 'NombreProducto',
       'short_name', 'brand', 'weight', 'pieces', 'Ruta_SAK_counts'],
      dtype='object')

In [30]:
# delete the last duplicate columns
#df_main.drop(list(df_main)[-1], axis=1, inplace=True)

In [31]:
#drop the unecessary columns for prediction

dropColumns = ["Agencia_ID", "Ruta_SAK", "Cliente_ID", "Producto_ID", "State", "Address", "NombreProducto"]
df_main.drop(dropColumns, axis=1, inplace=True)

In [32]:
# Rename the columns
renameColumns = {"Semana":"Week", 
                 "Venta_uni_hoy":"SalesUnitWeek",
                 "Venta_hoy": "SalesPesosWeek",
                 "Dev_proxima": "RetrunsPesosNextWeek",
                 "Dev_uni_proxima": "ReturnsUnitNextWeek",
                 "Demanda_uni_equil": "AjustedDemand",
                 "NombreCliente": "typeClient"
                 }
df_main.rename(columns=renameColumns, inplace=True)

In [33]:
#change the target from regression problem to Binary problem
df_main["AjustedDemand"] = df_main['AjustedDemand'].apply(lambda x: 0 if x==0 or np.log10(x) <0.56 else 1)

In [34]:
#produtos que foram vendidos, mas nao retornados
df_main["ReturnUnitMissing"] = df_main["SalesUnitWeek"] - df_main["ReturnsUnitNextWeek"]
df_main["ReturnPesosMissing"] = df_main["SalesPesosWeek"] - df_main["RetrunsPesosNextWeek"]

In [35]:
# Corrulation between target and others columns
corr_matrix = df_main.corr()
corr_matrix["AjustedDemand"].sort_values(ascending=False)

AjustedDemand           1.000000
ReturnUnitMissing       0.266096
SalesUnitWeek           0.262159
SalesPesosWeek          0.172192
ReturnPesosMissing      0.172138
Canal_ID                0.124718
Latitude                0.020015
pieces                 -0.001936
Week                   -0.005093
RetrunsPesosNextWeek   -0.007559
Longitude              -0.010470
ReturnsUnitNextWeek    -0.018341
Ruta_SAK_counts        -0.088658
weight                 -0.106641
Name: AjustedDemand, dtype: float64

In [36]:
df_main.to_csv("df_clean.csv", index=False)